In [1]:
import requests
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [2]:
def get(path, params=None):
    r = requests.get(path, params=params, headers=headers)
    r.raise_for_status()
    if (r.headers['content-type'] == 'application/json'):
        return r.json()
    
    if ('content-disposition' in r.headers):
        filename = r.headers['content-disposition'].split("filename=")[1]
        with open(filename, 'wb') as f:
            f.write(r.content)
        return filename
    return r


In [3]:
baseUrl = 'http://www.illustris-project.org/api/'
headers = {"api-key":"79d3daabdc8c4290395780680a4e1107"}

In [ ]:
url = "http://www.illustris-project.org/api/Illustris-1/snapshots/z=0/subhalos/"
subhalos = get(url)
subhalos['count']




In [ ]:
# first convert log solar masses into group catalog units
mass_min = 10**11.9 / 1e10 * 0.704
mass_max = 10**12.1 / 1e10 * 0.704

# form the search_query string by hand for once
search_query = "?mass__gt=" + str(mass_min) + "&mass__lt=" + str(mass_max)

# form the url and make the request
url = "http://www.illustris-project.org/api/Illustris-1/snapshots/135/subhalos/" + search_query
subhalos = get(url)
subhalos['count']


In [ ]:
len(subhalos['results'])

In [ ]:
ids = [ subhalos['results'][i]['id'] for i in range(len(subhalos['results'])) ]
ids


In [ ]:
sub_count = 1
plt.figure(figsize=[15,15])

for id in ids:
    url = "http://www.illustris-project.org/api/Illustris-1/snapshots/135/subhalos/" + str(id)
    sub = get(url)
    
    # it is of course possible this data product does not exist for all requested subhalos
    if 'stellar_mocks' in sub['supplementary_data']: 
        # download PNG image, the version which includes all stars in the FoF halo (try replacing 'fof' with 'gz')
        png_url = sub['supplementary_data']['stellar_mocks']['image_gz']
        response = get(png_url)
        
        # make plot a bit nicer
        plt.subplot(1,len(ids),sub_count)
        plt.text(0,-20,"ID="+str(id),color='blue')
        plt.gca().axes.get_xaxis().set_ticks([])
        plt.gca().axes.get_yaxis().set_ticks([])
        sub_count += 1
        
        # plot the PNG binary data directly, without actually saving a .png file
        file_object = StringIO(response.content)
        plt.imsave("haloID_"+str(id)+".png", mpimg.imread(file_object))
        #plt.imshow(mpimg.imread(file_object))